Beginner's Hypothesis

Although I tried many ways to get the best version of the model, I am showcasing the ones that provide good accuracy by training them in less time and consuming fewer computational resources
Such as:
1. I trained the motion and power on SGDClassifier in sklearn which used less computational resources and gave the best accuracy
2. Trained summary and element on 2D CNN as this the best way between the accuracy and computational power
3. Trained the speed over 3D CNN as there was no other option to get this much accuracy

Making the data ready for training

Here we are taking input from the train.csv to the pandas dataframe

In [90]:
import pandas as pd
file_path = 'BH25/Training_Data/train.csv' #Path to the training file
data = pd.read_csv(file_path)
print(data.info()) #For analysing data
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   video_id       10000 non-null  int64  
 1   element        10000 non-null  object 
 2   motion         10000 non-null  object 
 3   power          10000 non-null  object 
 4   speed          10000 non-null  float64
 5   video_summary  10000 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 468.9+ KB
None
   video_id element  motion power  speed          video_summary
0         1    Erde  linear   rot    9.6     (3.8147, 31.94809)
1         2   Feuer     shm  grin    9.6   (26.70288, -4.29153)
2         3   Feuer  random  geld    9.6      (3.8147, 8.58307)
3         4    Erde  zigzag  lila    6.9  (-24.79553, -0.95367)
4         5    Erde  linear  lila    9.6    (7.62939, 22.88818)


Checking the number of unique elements

In [91]:
data.groupby('power')['power'].agg('count') #Grouping the features with their numbers

power
baau     202
blau    1011
blru     202
brun     199
btau     203
gaun     199
gela     192
gelb     960
geld     192
gele     192
gelt     192
got      206
grin     199
grun     993
grus     198
helb     192
lela     201
liga     201
lila    1004
lili     201
lisa     201
lula     201
not      206
plau     202
rat      206
rod      207
rot     1031
rut      206
slau     202
trun     199
Name: power, dtype: int64

In [92]:
unique_elements = data['power'].unique()
print(len(unique_elements))

30


During data exploration and analysis, I learned that the data features correspond to different attributes, such as elements related to shape and power associated with color

Also, got some error in the data so fixed it as shown below

In [93]:
replacements = {
    'slau': 'blau', 'baau': 'blau', 'blru': 'blau', 'plau' : 'blau', 'btau' : 'blau',
    'trun' : 'grun', 'brun' : 'grun', 'gaun': 'grun', 'grin':'grun', 'grus' : 'grun',
    'helb' : 'gelb', 'gela' : 'gelb', 'geld' : 'gelb', 'gele' : 'gelb', 'gelt' : 'gelb',
    'rut' : 'rot', 'got' : 'rot', 'not' : 'rot', 'rat' : 'rot', 'rod' : 'rot',
    'lela' : 'lila', 'liga' : 'lila', 'lili' : 'lila', 'lisa' : 'lila', 'lula' : 'lila'
}
data2 = data
data2['power'] = data2['power'].replace(replacements)
unique_elements2 = data['power'].unique()
print(unique_elements2)

['rot' 'grun' 'gelb' 'lila' 'blau']


In [94]:
data2.groupby('power')['power'].agg('count')

power
blau    2022
gelb    1920
grun    1987
lila    2009
rot     2062
Name: power, dtype: int64

In [6]:
data2.groupby('motion')['motion'].agg('count')

motion
circular    2048
linear      1928
random      2004
shm         2027
zigzag      1993
Name: motion, dtype: int64

Separating the values into summary_x and summary_y and dropping the unnecessory video_summary column

In [7]:
data3 =data2
data3[['summary_x', 'summary_y']] = data3['video_summary'].str.extract(r'\((-?\d+\.?\d*),\s*(-?\d+\.?\d*)\)').astype(float)
print(data3.info())
print(data3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   video_id       10000 non-null  int64  
 1   element        10000 non-null  object 
 2   motion         10000 non-null  object 
 3   power          10000 non-null  object 
 4   speed          10000 non-null  float64
 5   video_summary  10000 non-null  object 
 6   summary_x      10000 non-null  float64
 7   summary_y      10000 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 625.1+ KB
None
   video_id element  motion power  speed          video_summary  summary_x  \
0         1    Erde  linear   rot    9.6     (3.8147, 31.94809)    3.81470   
1         2   Feuer     shm  grun    9.6   (26.70288, -4.29153)   26.70288   
2         3   Feuer  random  gelb    9.6      (3.8147, 8.58307)    3.81470   
3         4    Erde  zigzag  lila    6.9  (-24.79553, -0.95367)  -24.795

In [8]:
data3.drop(columns= ['video_summary'], inplace= True)
print(data3.info())
print(data3.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   video_id   10000 non-null  int64  
 1   element    10000 non-null  object 
 2   motion     10000 non-null  object 
 3   power      10000 non-null  object 
 4   speed      10000 non-null  float64
 5   summary_x  10000 non-null  float64
 6   summary_y  10000 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 547.0+ KB
None
   video_id element  motion power  speed  summary_x  summary_y
0         1    Erde  linear   rot    9.6    3.81470   31.94809
1         2   Feuer     shm  grun    9.6   26.70288   -4.29153
2         3   Feuer  random  gelb    9.6    3.81470    8.58307
3         4    Erde  zigzag  lila    6.9  -24.79553   -0.95367
4         5    Erde  linear  lila    9.6    7.62939   22.88818


Defining 3 types of preprocessing functions:
1. For Video training(speed)
2. For taking the image out of the video and train on it (summary,element)
3. For motion and power

In [9]:
import cv2
import numpy as np

def preprocess_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frames = []
    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # Read the next frame
        # If no frame is returned, end of video is reached
        if not ret:
            break
         # Resize the frame to 64x64 pixels
        frame = cv2.resize(frame, (64, 64))
        # Append the processed frame to the list
        frames.append(frame)
    # Release the video capture object
    cap.release()
    frames = np.array(frames) # Convert the list of frames to a NumPy array
    frames = frames / 255.0 # Normalize the pixel values to the range [0, 1]

    return frames

In [10]:
import cv2
import numpy as np

def preprocess_video_2(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frames = []
    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # Read the next frame
        # If no frame is returned, end of video is reached
        if not ret:
            break
         # Resize the frame to 64x64 pixels
        frame = cv2.resize(frame, (64, 64))
        # Append the processed frame to the list
        frames.append(frame)
    # Release the video capture object
    cap.release()
    frames = np.array(frames) # Convert the list of frames to a NumPy array
    frames = frames / 255.0 # Normalize the pixel values to the range [0, 1]

    return frames[1]

In [11]:
import cv2
import numpy as np

def preprocess_video_3(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    frames = []
    # Loop through each frame in the video
    while True:
        ret, frame = cap.read() # Read the next frame
        # If no frame is returned, end of video is reached
        if not ret:
            break
         # Resize the frame to 64x64 pixels
        frame = cv2.resize(frame, (64, 64))
        # Append the processed frame to the list
        frames.append(frame)
    # Release the video capture object
    cap.release()
    frames = np.array(frames) # Convert the list of frames to a NumPy array

    return frames.flatten() # Convert to (20*64*64*3)

Performing label encoding and saving the label encoders for later use

In [12]:
from sklearn.preprocessing import LabelEncoder  # Import LabelEncoder
import copy  # Import copy for deep copying

data4 = copy.deepcopy(data3)  # Create a deep copy of the original dataset

label_encoder = {}  # Initialize an empty dictionary to store label encoders

categorical_columns = ['element', 'motion', 'power']  # List of categorical columns

for col in categorical_columns:  # Loop through each categorical column
    le = LabelEncoder()  # Initialize a LabelEncoder for the current column
    data4[col] = le.fit_transform(data4[col])  # Fit and transform the column
    label_encoder[col] = le  # Store the fitted label encoder in the dictionary

del data3, data2, data  # Delete the original datasets to free up memory

data4.head()  # Display the first few rows of the transformed dataset

Saving the label encoders to use afterwards

In [13]:
import joblib  # Import joblib for saving and loading models and encoders

# Loop through each categorical column and save the corresponding label encoder
for col in categorical_columns:
    joblib.dump(label_encoder[col], f"label_encoder_{col}.joblib")  # Save label encoder to a file

print("saved")  # Print confirmation message after saving

saved


Defining 2 classes VideoDataGenerator(for getting the input of the 3D CNN) and DataGenerator(for getting the input of the 2D CNN)

In [38]:
import os  # Import os for file path operations
import numpy as np  # Import NumPy for numerical operations
import cv2  # Import OpenCV for video processing
import tensorflow as tf  # Import TensorFlow

class VideoDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, column, video_dir, batch_size=32, num_frames=20, validation_split=0.2, subset=None, shuffle=True):
        """
        Args:
            dataframe (pd.DataFrame): Contains video names and labels.
            video_dir (str): Directory containing video files.
            batch_size (int): Number of samples per batch.
            img_size (tuple): Target image size (height, width).
            num_frames (int): Number of frames to sample from each video.
            shuffle (bool): Whether to shuffle the dataset after each epoch.
        """
        self.dataframe = dataframe  # DataFrame containing video names and labels
        self.video_dir = video_dir  # Directory containing video files
        self.batch_size = batch_size  # Number of samples per batch
        self.num_frames = num_frames  # Number of frames to sample from each video
        self.shuffle = shuffle  # Whether to shuffle the dataset after each epoch
        self.subset = subset  # Subset type (training, validation, testing)
        self.column = column  # Column containing labels

        # Split dataset into training and validation subsets
        total_samples = len(dataframe)
        split_index = int(total_samples * (1 - validation_split))

        if subset == "training":
            self.dataframe = dataframe.iloc[:split_index]
        elif subset == "validation":
            self.dataframe = dataframe.iloc[split_index:]
        elif subset == "testing":
            pass  # No split for testing

        self.indexes = np.arange(len(self.dataframe))  # Initialize indexes
        self.on_epoch_end()  # Shuffle dataset if required

    def on_epoch_end(self):
        """Shuffle dataset at the end of each epoch."""
        if self.shuffle:
            np.random.shuffle(self.indexes)  # Shuffle indexes

    def __len__(self):
        """Return the number of batches per epoch."""
        return int(np.ceil(len(self.dataframe) / self.batch_size))  # Calculate number of batches

    def __getitem__(self, index):
        """Generate one batch of data."""
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]  # Get batch indexes
        batch_data = self.dataframe.iloc[batch_indexes]  # Get batch data

        videos = []
        for i, row in batch_data.iterrows():
            try:
                video_id_str = str(int(row['video_id']))  # Ensure video_id is a string
                video_path = os.path.join(self.video_dir, video_id_str + '.mp4')  # Construct video path
                video = preprocess_video(video_path)  # Preprocess video
                if video is not None:
                    videos.append(video)  # Append processed video to list
            except Exception as e:
                print(e)  # Print any exceptions

        # Add labels (adjust according to your dataset)
        labels = np.array(batch_data[self.column])  # Get labels for the batch
        return np.array(videos), labels  # Return videos and labels as arrays

In [15]:
import os  # Import os for file path operations
import numpy as np  # Import NumPy for numerical operations
import cv2  # Import OpenCV for video processing
import tensorflow as tf  # Import TensorFlow

class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, column, video_dir, batch_size=32, num_frames=20, validation_split=0.2, subset=None, shuffle=False):
        """
        Args:
            dataframe (pd.DataFrame): Contains video names and labels.
            video_dir (str): Directory containing video files.
            batch_size (int): Number of samples per batch.
            img_size (tuple): Target image size (height, width).
            num_frames (int): Number of frames to sample from each video.
            shuffle (bool): Whether to shuffle the dataset after each epoch.
        """
        self.dataframe = dataframe  # DataFrame containing video names and labels
        self.video_dir = video_dir  # Directory containing video files
        self.batch_size = batch_size  # Number of samples per batch
        self.num_frames = num_frames  # Number of frames to sample from each video
        self.shuffle = shuffle  # Whether to shuffle the dataset after each epoch
        self.subset = subset  # Subset type (training, validation, testing)
        self.column = column  # Column containing labels

        # Split dataset into training and validation subsets
        total_samples = len(dataframe)  # Total number of samples
        split_index = int(total_samples * (1 - validation_split))  # Index to split data

        if subset == "training":
            self.dataframe = dataframe.iloc[:split_index]  # Training subset
        elif subset == "validation":
            self.dataframe = dataframe.iloc[split_index:]  # Validation subset
        elif subset == "testing":
            pass  # No split for testing

        self.indexes = np.arange(len(self.dataframe))  # Initialize indexes
        self.on_epoch_end()  # Shuffle dataset if required

    def on_epoch_end(self):
        """Shuffle dataset at the end of each epoch."""
        if self.shuffle:
            np.random.shuffle(self.indexes)  # Shuffle indexes

    def __len__(self):
        """Return the number of batches per epoch."""
        return int(np.ceil(len(self.dataframe) / self.batch_size))  # Calculate number of batches

    def __getitem__(self, index):
        """Generate one batch of data."""
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]  # Get batch indexes
        batch_data = self.dataframe.iloc[batch_indexes]  # Get batch data

        videos = []
        for i, row in batch_data.iterrows():  # Iterate through each row in the batch
            try:
                video_id_str = str(int(row['video_id']))  # Ensure video_id is a string
                video_path = os.path.join(self.video_dir, video_id_str + '.mp4')  # Construct video path
                video = preprocess_video_2(video_path)  # Preprocess video
                if video is not None:
                    videos.append(video)  # Append processed video to list
            except Exception as e:
                print(e)  # Print any exceptions

        # Add labels (adjust according to your dataset)
        labels = np.array(batch_data[self.column])  # Get labels for the batch
        return np.array(videos), labels  # Return videos and labels as arrays

Training the model for elements

In [16]:
# Set the directory for training videos
video_dir = 'BH25\\Training_Data\\Train_Videos'

# Create a data generator for the training subset
train_data_gen = DataGenerator(
    data4,         # The data source
    'element',     # The label or target variable
    video_dir,     # The directory containing the videos
    subset='training' # Specify that this is the training subset
)

# Create a data generator for the validation subset
val_data_gen = DataGenerator(
    data4,           # The data source
    'element',       # The label or target variable
    video_dir,       # The directory containing the videos
    subset='validation' # Specify that this is the validation subset
)

I also tried identifying the elements(shapes) using OpenCV (but the accuracy was a bit lower than the one achieved by training a 2D CNN), so I trained a model using a 2D CNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Initialize a sequential model
model_element = Sequential([
    # First convolutional layer
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    # Second convolutional layer
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    # Third convolutional layer
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    # Flatten the output
    Flatten(),
    # Fully connected layer
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Output layer with softmax activation for multi-class classification
    Dense(5, activation='softmax')
])

# Compile the model with Adam optimizer and sparse categorical cross-entropy loss
model_element.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [18]:
model_element.fit(
    train_data_gen,
    validation_data= val_data_gen,
    epochs = 10
)

c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 83s 322ms/step - accuracy: 0.5296 - loss: 2.0450 - val_accuracy: 0.2135 - val_loss: 59.5493
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 69s 277ms/step - accuracy: 0.8818 - loss: 0.3016 - val_accuracy: 0.2895 - val_loss: 40.4940
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 77s 309ms/step - accuracy: 0.9557 - loss: 0.1245 - val_accuracy: 0.9590 - val_loss: 0.1089
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 78s 314ms/step - accuracy: 0.9752 - loss: 0.0659 - val_accuracy: 0.9640 - val_loss: 0.1022
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 76s 306ms/step - accuracy: 0.9892 - loss: 0.0441 - val_accuracy: 0.9000 - val_loss: 0.3774
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 79s 316ms/step - accuracy: 0.9887 - loss: 0.0340 - val_accuracy: 0.9450 - val_loss: 0.2446
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 302ms/step - accuracy: 0.9884 - loss: 0.0379 - val_accuracy: 0.9400 - val_loss: 0.3006
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 78s 313ms/step - accuracy: 0.9912 - loss:

In [19]:
model_element.save("model_element.h5")

Predicting the element of video

In [20]:
def predict_element(video_path):
    inputs = tf.expand_dims(preprocess_video_2(video_path), axis=0)
    predict_array = model_element.predict(inputs)
    predict_element = label_encoder['element'].inverse_transform(tf.argmax(predict_array,axis=1))
    print(predict_element[0])
    return predict_element[0]

In [21]:
video_path = 'BH25/Training_Data/Train_Videos/2.mp4'
predict_element(video_path)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
Feuer


'Feuer'

In [22]:
import os
elements = []
video_ids = []
test_dir = 'BH25/Testing_Data'
for video_id in sorted(
    [f for f in os.listdir(test_dir) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_dir, video_id)
    video_ids.append(video_id.split('.')[0])
    elements.append(predict_element(video_path))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Wasser
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Feuer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Luft
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Feuer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Wasser
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Feuer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Wasser
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Wasser
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Erde
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Erde
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Wasser
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Feuer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Luft
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Boden
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30m

In [23]:
print(elements[:5])
print(video_ids[:5]) #Just for checking everything is working fine

['Boden', 'Boden', 'Wasser', 'Feuer', 'Luft']
['1', '2', '3', '4', '5']


Now training the model for motion and power

In [ ]:
# Extracting the video features from video
video_features = []  # List to store features extracted from videos
X_element = []  # Placeholder for future use

# Iterate over each row in the dataset
for idx, row in data4.iterrows():
    # Construct the video file path
    video_path = f"BH25/Training_Data/Train_Videos/{str(int(row['video_id']))}.mp4"
    # Preprocess the video and append the features to the list
    video_features.append(preprocess_video_3(video_path))

# Convert the list of video features to a NumPy array
video_features = np.array(video_features)  # Shape: (num_samples, 20*64*64*3)
print(video_features.shape)  # Print the shape of the resulting array

(10000, 245760)


In [28]:
from sklearn.model_selection import train_test_split
# splitting into train and validation
y_categorical = data4[categorical_columns]
X_train_cat, X_val_cat, y_train_cat, y_val_cat = train_test_split(video_features, y_categorical, test_size=0.2, random_state=42)

Here, I used an SGDClassifier because it is far more efficient than a CNN and can be trained on a large dataset without causing RAM issues, unlike Logistic Regression

In [29]:
from sklearn.linear_model import SGDClassifier

models_categorical = {}  # Dictionary to store the models for each categorical column
batch_size = 1000  # Define the batch size for incremental learning

for col in categorical_columns:
    if col == 'element':
        continue  # Skip the 'element' column
    else:
        # Initialize the SGDClassifier for the current column
        models_categorical[col] = SGDClassifier(loss='log_loss')
        
        # Perform incremental learning in batches
        for i in range(0, len(X_train_cat), batch_size): 
            X_batch = X_train_cat[i:i + batch_size]  # Extract a batch of training data
            y_batch = y_train_cat[i:i + batch_size]  # Extract the corresponding labels
            
            # Incrementally train the model on the current batch
            models_categorical[col].partial_fit(X_batch, y_batch[col], classes=np.unique(y_batch[col]))

Checking the accuracy on validation set

In [30]:
from sklearn.metrics import accuracy_score

categorical_accuracy = {}  # Dictionary to store accuracy for each categorical column

for col in categorical_columns:
    if col == 'element':
        continue  # Skip the 'element' column
    else:
        # Predict the validation data for the current column
        y_categorical_pred = models_categorical[col].predict(X_val_cat)
        
        # Calculate accuracy for the current column
        accuracy = accuracy_score(y_val_cat[col], y_categorical_pred)
        
        # Store the accuracy in the dictionary
        categorical_accuracy[col] = accuracy

# Print the dictionary with accuracy for each column
print(categorical_accuracy)

{'motion': 1.0, 'power': 0.998}


In [31]:
import joblib
# Saving the models

# Loop through each categorical column
for col in categorical_columns:
    if col == 'element':
        continue  # Skip the 'element' column
    else:
        # Save the model for the current column using joblib
        joblib.dump(models_categorical[col], f"model_categorical_{col}.joblib")

In [32]:
import os
# Loading the features of the videos in testing_data
test_folder = 'BH25/Testing_Data'

test_features = []
video_ids = []
test_features_element =[]

# The below function finds all the video ids in the test folder and sorts them, 
# and then stores their features in test_features

for video_id in sorted(
    [f for f in os.listdir(test_folder) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_folder, video_id)
    video_ids.append(video_id.split('.')[0]) 
    test_features.append(preprocess_video_3(video_path))

test_features = np.array(test_features)  # Shape: (num_test_samples, 20*64*64*3)

Now, prdicting the motion and power of the videos in testing folder

In [33]:
y_categorical = {}  # Dictionary to store predictions for each categorical column

# Loop through each categorical column
for col in categorical_columns:
    if col == 'element':
        continue 
    else:
        # Predict the test features using the model for the current column
        prediction = models_categorical[col].predict(test_features)
        
        # Inverse transform the predictions to get the original labels
        y_categorical[col] = label_encoder[col].inverse_transform(prediction)

In [34]:
print(y_categorical)

{'motion': array(['zigzag', 'zigzag', 'random', ..., 'random', 'linear', 'circular'],
      dtype=object), 'power': array(['rot', 'lila', 'blau', ..., 'gelb', 'gelb', 'lila'], dtype=object)}


Now training it for speed

In [40]:
# Set the directory for training videos
video_dir = 'BH25\\Training_Data\\Train_Videos'

# Create a video data generator for the training subset
train_data_gen = VideoDataGenerator(
    data4,         # Data source
    'speed',       # Label or target variable
    video_dir,     # Directory containing the videos
    subset='training' # Specify that this is the training subset
)

# Create a video data generator for the validation subset
val_data_gen = VideoDataGenerator(
    data4,           # Data source
    'speed',         # Label or target variable
    video_dir,       # Directory containing the videos
    subset='validation' # Specify that this is the validation subset
)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization

# Initialize a sequential model
model_speed = Sequential([
    # First 3D convolutional layer
    Conv3D(32, (3, 3, 3), activation='relu', input_shape=(20, 64, 64, 3)),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    # Second 3D convolutional layer
    Conv3D(64, (3, 3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    # Third 3D convolutional layer
    Conv3D(128, (3, 3, 3), activation='relu'),
    BatchNormalization(),
    # Flatten the output
    Flatten(),
    # Fully connected layer
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Output layer with a single unit (for regression)
    Dense(1)
])

# Compile the model with Adam optimizer and mean squared error loss
model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [42]:
model_speed.fit(
    train_data_gen,
    validation_data= val_data_gen,
    epochs = 10
)

c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 212s 839ms/step - loss: 15.8681 - mae: 2.9049 - val_loss: 7508.1948 - val_mae: 83.7328
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 201s 803ms/step - loss: 3.8884 - mae: 1.5726 - val_loss: 292.9991 - val_mae: 12.1004
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 195s 781ms/step - loss: 3.1431 - mae: 1.3939 - val_loss: 0.6226 - val_mae: 0.5949
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 187s 748ms/step - loss: 2.5949 - mae: 1.2751 - val_loss: 0.6178 - val_mae: 0.6175
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 188s 753ms/step - loss: 2.3985 - mae: 1.2385 - val_loss: 0.7105 - val_mae: 0.6707
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 185s 742ms/step - loss: 2.2311 - mae: 1.1833 - val_loss: 0.6593 - val_mae: 0.6222
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 183s 732ms/step - loss: 2.1952 - mae: 1.1664 - val_loss: 0.7111 - val_mae: 0.6585
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 184s 734ms/step - loss: 2.1775 - mae: 1.1747 - val_loss: 0.7916 - val_mae: 0.7361
Epoch 9/10
250/2

In [43]:
model_speed.save("model_speed.h5")

Predicting the speed feature of the videos in testing data

In [44]:
import os
import tensorflow as tf

test_dir = 'BH25/Testing_Data'  # Directory containing test videos
video_ids = []  # List to store video IDs
predict_speed = []  # List to store predicted speeds

# Loop through all .mp4 files in the test directory, sorted by numerical order
for video_id in sorted([f for f in os.listdir(test_dir) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_dir, video_id)  # Construct the full video path
    video_ids.append(video_id.split('.')[0])  # Extract and store the video ID
    inputs = tf.expand_dims(preprocess_video(video_path), axis=0)  # Preprocess video and add a batch dimension
    predict_speed.append(model_speed.predict(inputs))  # Predict the speed and append to the list


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

In [45]:
print(predict_speed) # Saving the speed model

[array([[7.9817576]], dtype=float32), array([[7.7986937]], dtype=float32), array([[8.5617075]], dtype=float32), array([[6.6312704]], dtype=float32), array([[10.346007]], dtype=float32), array([[6.6257014]], dtype=float32), array([[7.117166]], dtype=float32), array([[8.5987835]], dtype=float32), array([[6.2559004]], dtype=float32), array([[9.801225]], dtype=float32), array([[8.071639]], dtype=float32), array([[7.2055464]], dtype=float32), array([[6.41699]], dtype=float32), array([[9.198818]], dtype=float32), array([[8.9914665]], dtype=float32), array([[9.571413]], dtype=float32), array([[10.451032]], dtype=float32), array([[8.726241]], dtype=float32), array([[9.3190565]], dtype=float32), array([[8.821389]], dtype=float32), array([[9.911729]], dtype=float32), array([[9.734284]], dtype=float32), array([[6.8796988]], dtype=float32), array([[8.896686]], dtype=float32), array([[6.888896]], dtype=float32), array([[6.828529]], dtype=float32), array([[8.508743]], dtype=float32), array([[9.79614

Since I was unable to determine how this attribute is found from the video feature, I tried three possible methods:
1. Linear Regression
2. 2D CNN
3. 3D CNN

Didn't upload the regression one as it was not the successful one and taking too much memory to train on the whole dataset

In [46]:
video_dir = 'BH25\\Training_Data\\Train_Videos'
train_data_gen = DataGenerator(
    data4,
    'summary_x',
    video_dir,
    subset = 'training'
)

val_data_gen = DataGenerator(
    data4,
    'summary_x',
    video_dir,
    subset = 'validation'
)

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model_summary_x = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1) 
])

model_summary_x.compile(optimizer='adam', loss='mse', metrics=['mae'])

c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [48]:
model_summary_x.fit(
    train_data_gen,
    validation_data= val_data_gen,
    epochs = 10
)

c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 76s 296ms/step - loss: 297.2528 - mae: 13.6821 - val_loss: 309.5639 - val_mae: 13.6586
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 70s 282ms/step - loss: 258.7744 - mae: 12.6562 - val_loss: 266.6958 - val_mae: 12.8272
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 71s 284ms/step - loss: 246.3934 - mae: 12.4632 - val_loss: 280.9667 - val_mae: 13.1446
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 73s 293ms/step - loss: 220.2286 - mae: 11.6693 - val_loss: 309.6212 - val_mae: 13.7761
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 70s 279ms/step - loss: 190.4449 - mae: 10.8652 - val_loss: 357.6566 - val_mae: 14.7591
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 74s 296ms/step - loss: 127.9791 - mae: 8.7961 - val_loss: 346.5640 - val_mae: 14.6737
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 288ms/step - loss: 98.3310 - mae: 7.6674 - val_loss: 353.0437 - val_mae: 14.9098
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 290ms/step - loss: 78.3732 - mae: 6.7326 - val_loss: 1467.0996 - val_m

In [49]:
model_summary_x.save("model_summary_x.h5") # Saving the model

Training for summary_y

In [50]:
video_dir = 'BH25\\Training_Data\\Train_Videos'
train_data_gen = DataGenerator(
    data4,
    'summary_y',
    video_dir,
    subset = 'training'
)

val_data_gen = DataGenerator(
    data4,
    'summary_y',
    video_dir,
    subset = 'validation'
)

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model_summary_y = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1)
])

model_summary_y.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [52]:
model_summary_x.fit(
    train_data_gen,
    validation_data= val_data_gen,
    epochs = 10
)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 287ms/step - loss: 257.8670 - mae: 12.5652 - val_loss: 223.9574 - val_mae: 11.9316
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 69s 275ms/step - loss: 212.6426 - mae: 11.6158 - val_loss: 227.0409 - val_mae: 12.0119
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 67s 269ms/step - loss: 211.8932 - mae: 11.5705 - val_loss: 227.7461 - val_mae: 11.9777
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 72s 290ms/step - loss: 206.3074 - mae: 11.3814 - val_loss: 263.5427 - val_mae: 12.9298
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 75s 298ms/step - loss: 198.6938 - mae: 11.2243 - val_loss: 231.8264 - val_mae: 12.1757
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 73s 293ms/step - loss: 180.9154 - mae: 10.6198 - val_loss: 469.9232 - val_mae: 17.1730
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 74s 295ms/step - loss: 151.9989 - mae: 9.5978 - val_loss: 258.7771 - val_mae: 12.7534
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 74s 298ms/step - loss: 123.9203 - mae: 8.5712 - val_loss: 238.6332 - val

In [53]:
model_summary_y.save("model_summary_y.h5") # Saving the model

Predicting the summary column(summary_x and summary_y)

In [54]:
import os
import tensorflow as tf

test_dir = 'BH25/Testing_Data'  # Define testing data directory

video_ids = []
predict_summary_x = []
predict_summary_y = []

for video_id in sorted(
    [f for f in os.listdir(test_dir) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_dir, video_id)  # Construct video file path
    inputs = tf.expand_dims(preprocess_video_2(video_path), axis=0)  # Preprocess and add dimension
    predict_summary_x.append(model_summary_x.predict(inputs))  # Append predictions for model X
    predict_summary_y.append(model_summary_y.predict(inputs))  # Append predictions for model Y


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━

In [55]:
print(predict_summary_x)

[array([[0.24815692]], dtype=float32), array([[0.5315128]], dtype=float32), array([[-4.1715307]], dtype=float32), array([[2.0711925]], dtype=float32), array([[-3.0207071]], dtype=float32), array([[-0.3839182]], dtype=float32), array([[-2.4579792]], dtype=float32), array([[-2.3283284]], dtype=float32), array([[-0.32527086]], dtype=float32), array([[-0.34182897]], dtype=float32), array([[1.2359747]], dtype=float32), array([[-1.9054555]], dtype=float32), array([[6.1563706]], dtype=float32), array([[1.6026657]], dtype=float32), array([[-4.9761653]], dtype=float32), array([[-3.6248748]], dtype=float32), array([[0.9831631]], dtype=float32), array([[-1.0023547]], dtype=float32), array([[5.482228]], dtype=float32), array([[2.582477]], dtype=float32), array([[5.1630826]], dtype=float32), array([[0.04817816]], dtype=float32), array([[1.4765631]], dtype=float32), array([[-6.676467]], dtype=float32), array([[-4.574901]], dtype=float32), array([[-7.6144443]], dtype=float32), array([[4.1824307]], dt

In [56]:
print(predict_summary_y)

[array([[0.02594363]], dtype=float32), array([[0.00651491]], dtype=float32), array([[0.02667546]], dtype=float32), array([[-0.00102132]], dtype=float32), array([[0.01992751]], dtype=float32), array([[0.0061211]], dtype=float32), array([[-0.00357406]], dtype=float32), array([[0.00783666]], dtype=float32), array([[0.01021021]], dtype=float32), array([[0.00909527]], dtype=float32), array([[-0.02108204]], dtype=float32), array([[-0.00036039]], dtype=float32), array([[0.00809838]], dtype=float32), array([[0.00820828]], dtype=float32), array([[0.01040266]], dtype=float32), array([[-0.00930668]], dtype=float32), array([[0.00346148]], dtype=float32), array([[-0.01357868]], dtype=float32), array([[0.01906316]], dtype=float32), array([[0.01516828]], dtype=float32), array([[0.00576454]], dtype=float32), array([[0.01707663]], dtype=float32), array([[0.02323344]], dtype=float32), array([[0.00486114]], dtype=float32), array([[0.00798661]], dtype=float32), array([[0.00352587]], dtype=float32), array(

In [63]:
# Converting all continuous values to numpy array and making them 1 dimensional
predict_summary_x = np.array(predict_summary_x)
predict_summary_y = np.array(predict_summary_y)
predict_speed = np.array(predict_speed)
predict_summary_x = predict_summary_x.flatten()
predict_summary_y = predict_summary_y.flatten()
predict_speed = predict_speed.flatten()

Converting all the attributes to the dataframe

In [64]:
# Compiling the dataframe
output_df = pd.DataFrame({
    'video_id': video_ids,                      
    'element': elements,  
    'motion': y_categorical['motion'],
    'power': y_categorical['power'],
    'speed': predict_speed,               
    'video_summary': list(zip(predict_summary_x, predict_summary_y)) 
})

Seeing every thing worked well

In [65]:
output_df.head()

,video_id,element,motion,power,speed,video_summary
0,1,Boden,zigzag,rot,7.981758,"(0.24815692, 0.025943633)"
1,2,Boden,zigzag,lila,7.798694,"(0.5315128, 0.0065149055)"
2,3,Wasser,random,blau,8.561707,"(-4.1715307, 0.026675459)"
3,4,Feuer,circular,rot,6.631270,"(2.0711925, -0.0010213227)"
4,5,Luft,linear,blau,10.346007,"(-3.0207071, 0.019927507)"


In [66]:
output_df.to_csv('Submission1.csv', index=False)

It is just a test whether rounding off will increase the accuracy or not as all values are either 6.9 or 9.6 in training data

In [67]:
adjusted_predictions = np.where(
    np.abs(predict_speed - 6.9) < np.abs(predict_speed - 9.6),
    6.9,
    9.6
)

In [68]:
output_df_2 = pd.DataFrame({
    'video_id': video_ids,                      
    'element': elements,  
    'motion': y_categorical['motion'],
    'power': y_categorical['power'],
    'speed': adjusted_predictions,               
    'video_summary': list(zip(predict_summary_x, predict_summary_y)) 
})

In [69]:
output_df_2.head()

,video_id,element,motion,power,speed,video_summary
0,1,Boden,zigzag,rot,6.9,"(0.24815692, 0.025943633)"
1,2,Boden,zigzag,lila,6.9,"(0.5315128, 0.0065149055)"
2,3,Wasser,random,blau,9.6,"(-4.1715307, 0.026675459)"
3,4,Feuer,circular,rot,6.9,"(2.0711925, -0.0010213227)"
4,5,Luft,linear,blau,9.6,"(-3.0207071, 0.019927507)"


In [70]:
output_df_2.to_csv('Submission2.csv', index=False)

Also tried to train video summary in 3D CNN also

By mistake the data of the model_summary_x2 was deleted so here output is not shown

In [ ]:
video_dir = 'BH25\\Training_Data\\Train_Videos'
train_data_gen = VideoDataGenerator(
    data4,
    'summary_x',
    video_dir,
    subset = 'training'
)

val_data_gen = VideoDataGenerator(
    data4,
    'summary_x',
    video_dir,
    subset = 'validation'
)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization

# Initialize a sequential model
model_summary_x2 = Sequential([
    # First 3D convolutional layer
    Conv3D(32, (3, 3, 3), activation='relu', input_shape=(20, 64, 64, 3)),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    # Second 3D convolutional layer
    Conv3D(64, (3, 3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    # Third 3D convolutional layer
    Conv3D(128, (3, 3, 3), activation='relu'),
    BatchNormalization(),
    # Flatten the output
    Flatten(),
    # Fully connected layer
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Output layer with a single unit (for regression)
    Dense(1)
])

# Compile the model with Adam optimizer and mean squared error loss
model_summary_x2.compile(optimizer='adam', loss='mse', metrics=['mae'])


In [ ]:
model_summary_x2.fit(
    train_data_gen,
    validation_data= val_data_gen,
    epochs = 10
)

In [ ]:
model_summary_x2.save('model_summary_x2.h5')

In [76]:
video_dir = 'BH25\\Training_Data\\Train_Videos'
train_data_gen = VideoDataGenerator(
    data4,
    'summary_y',
    video_dir,
    subset = 'training'
)

val_data_gen = VideoDataGenerator(
    data4,
    'summary_y',
    video_dir,
    subset = 'validation'
)

In [77]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization

# Initialize a sequential model
model_summary_y2 = Sequential([
    # First 3D convolutional layer
    Conv3D(32, (3, 3, 3), activation='relu', input_shape=(20, 64, 64, 3)),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    # Second 3D convolutional layer
    Conv3D(64, (3, 3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling3D(pool_size=(2, 2, 2)),
    # Third 3D convolutional layer
    Conv3D(128, (3, 3, 3), activation='relu'),
    BatchNormalization(),
    # Flatten the output
    Flatten(),
    # Fully connected layer
    Dense(128, activation='relu'),
    Dropout(0.5),
    # Output layer with a single unit (for regression)
    Dense(1)  # Assuming this is for a regression task
])

# Compile the model with Adam optimizer and mean squared error loss
model_summary_y2.compile(optimizer='adam', loss='mse', metrics=['mae'])


c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [79]:
model_summary_y2.fit(
    train_data_gen,
    validation_data= val_data_gen,
    epochs = 10
)

Epoch 1/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 727ms/step - loss: 244.1978 - mae: 12.4085

c:\Users\hy608\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 210s 829ms/step - loss: 244.1342 - mae: 12.4071 - val_loss: 328.2110 - val_mae: 14.4904
Epoch 2/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 202s 809ms/step - loss: 217.5947 - mae: 11.7539 - val_loss: 238.0981 - val_mae: 12.3096
Epoch 3/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 216s 864ms/step - loss: 214.4097 - mae: 11.6922 - val_loss: 461.4809 - val_mae: 16.9743
Epoch 4/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 197s 790ms/step - loss: 219.2326 - mae: 11.8257 - val_loss: 8206.0195 - val_mae: 71.0606
Epoch 5/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 196s 783ms/step - loss: 216.8139 - mae: 11.8091 - val_loss: 4907.4185 - val_mae: 55.8475
Epoch 6/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 210s 840ms/step - loss: 219.4171 - mae: 11.7718 - val_loss: 225.7247 - val_mae: 11.9846
Epoch 7/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 204s 816ms/step - loss: 210.2363 - mae: 11.5181 - val_loss: 233.7391 - val_mae: 12.1567
Epoch 8/10
250/250 ━━━━━━━━━━━━━━━━━━━━ 192s 769ms/step - loss: 217.0831 - mae: 11.7146 - val_loss: 224.1884 - va

In [80]:
model_summary_y2.save('model_summary_y2.h5')

In [82]:
import os
import tensorflow as tf
test_dir = 'BH25/Testing_Data'
video_ids = []
predict_summary_x2 = []
predict_summary_y2 = []

for video_id in sorted(
    [f for f in os.listdir(test_dir) if f.endswith(".mp4")], key=lambda x: int(x.split('.')[0])):
    video_path = os.path.join(test_dir, video_id)
    video_ids.append(video_id.split('.')[0])
    inputs = tf.expand_dims(preprocess_video(video_path),axis=0)
    predict_summary_x2.append(model_summary_x2.predict(inputs))
    predict_summary_y2.append(model_summary_y2.predict(inputs))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━

In [83]:
predict_summary_x2 = np.array(predict_summary_x2)
predict_summary_y2 = np.array(predict_summary_y2)
predict_summary_x2 = predict_summary_x2.flatten()
predict_summary_y2 = predict_summary_y2.flatten()

In [84]:
# Compiling the dataframe
output_df_3 = pd.DataFrame({
    'video_id': video_ids,                      
    'element': elements,  
    'motion': y_categorical['motion'],
    'power': y_categorical['power'],
    'speed': predict_speed,               
    'video_summary': list(zip(predict_summary_x2, predict_summary_y2)) 
})

In [ ]:
output_df_3.to_csv("Submission3.csv",index=False) # Saving the csv file

Thank you for giving me the chance to work on such an amazing project